In [1]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [2]:
def simulation_run(run, M):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 150
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{run}_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{run}_py.csv', delimiter=",")
    
    # y, coefficients = generate_dgp_tvp_var(M, T, p, 1/2, 1/4, 4*1e-5, 1, 1/2, 1e-2)
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)

    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run} & M: {M} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M]

In [3]:
%%time
import time
import os
from multiprocessing import Pool, Array

# They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
import warnings
warnings.filterwarnings("ignore")

np.random.seed(12345)
n_iterations = 200

iterations_set = np.arange(1,n_iterations+1,1)
M = (2,5,10)
args_tuples = [x for sub_list in [[(n, m) for n in iterations_set] for m in M] for x in sub_list]

if __name__ == '__main__':
    
    pool = Pool()
    simulation_results = pool.starmap(simulation_run, args_tuples)

Run: 91 & M: 2 -> MSD: 0.030324315778541888 | MSFE: 0.874752719321771 | ALPL: 0.3177941312589253
Run: 51 & M: 2 -> MSD: 0.043250084544254865 | MSFE: 1.2916955810130792 | ALPL: 0.33913923966091103
Run: 41 & M: 2 -> MSD: 0.033389187763203314 | MSFE: 1.538801884707439 | ALPL: 0.18896764383444065
Run: 81 & M: 2 -> MSD: 0.0319735570685065 | MSFE: 0.9186263854236065 | ALPL: 0.2650768032194984
Run: 151 & M: 2 -> MSD: 0.0439750305352035 | MSFE: 1.2087215603309935 | ALPL: 0.2871660210647726
Run: 11 & M: 2 -> MSD: 0.025527920077458025 | MSFE: 1.3225420243578618 | ALPL: 0.3394043970002782
Run: 131 & M: 2 -> MSD: 0.02330671503760985 | MSFE: 1.4241414925114189 | ALPL: 0.19405762551595357
Run: 21 & M: 2 -> MSD: 0.038146057733247675 | MSFE: 1.6597998754974457 | ALPL: 0.173261000684929
Run: 61 & M: 2 -> MSD: 0.046635517024428 | MSFE: 2.7365242039227407 | ALPL: 0.15902394710041434
Run: 71 & M: 2 -> MSD: 0.040774586160022344 | MSFE: 0.980381538812382 | ALPL: 0.3831477727096618
Run: 141 & M: 2 -> MSD: 0.

In [4]:
T = 200
train = 150
p = 1
prior = 'lasso'

simulation_parameters = [T,p,train, 1/2, 1/4, 4*1e-5, 1, 1/2, 1e-2, 12345, prior]

dump_to_disk = [simulation_results, simulation_parameters]

with open(f'../simulations/statistics_{T}_{p}_{prior}_{n_iterations}_contemp.pkl', 'wb') as f:
        pickle.dump(dump_to_disk, f, pickle.HIGHEST_PROTOCOL)